In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score,auc,accuracy_score,classification_report,confusion_matrix

In [11]:
raw_data = pd.read_csv('pima-indians-diabetes1.csv')
raw_data.columns = ['pregnant number','pgc','dbp','sft','insulin','bmi','dpf','age','class']

In [17]:
print(raw_data.describe())
attributes = raw_data.iloc[:,:-1].values
labels = raw_data.iloc[:,-1].values
trainX, testX, trainY, testY  = train_test_split(attributes,labels,test_size=0.3,random_state=30,stratify=labels)

       pregnant number         pgc         dbp         sft     insulin  \
count       767.000000  767.000000  767.000000  767.000000  767.000000   
mean          3.842243  120.859192   69.101695   20.517601   79.903520   
std           3.370877   31.978468   19.368155   15.954059  115.283105   
min           0.000000    0.000000    0.000000    0.000000    0.000000   
25%           1.000000   99.000000   62.000000    0.000000    0.000000   
50%           3.000000  117.000000   72.000000   23.000000   32.000000   
75%           6.000000  140.000000   80.000000   32.000000  127.500000   
max          17.000000  199.000000  122.000000   99.000000  846.000000   

              bmi         dpf         age       class  
count  767.000000  767.000000  767.000000  767.000000  
mean    31.990482    0.471674   33.219035    0.348110  
std      7.889091    0.331497   11.752296    0.476682  
min      0.000000    0.078000   21.000000    0.000000  
25%     27.300000    0.243500   24.000000    0.000000

In [57]:
classifier = MLPClassifier(hidden_layer_sizes=(20,), max_iter=100)
classifier.fit(trainX,trainY)
predictions = classifier.predict(testX)

/Users/vatsal/opt/anaconda3/envs/DMML/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [59]:
print(classification_report(testY,predictions))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78       151
           1       0.59      0.49      0.53        80

    accuracy                           0.71       231
   macro avg       0.67      0.65      0.66       231
weighted avg       0.70      0.71      0.70       231

